
# GameGuru
Grupo: Omega
Integrantes:
- Jorge Alcalde Piñeiro
- Ismael Salgado López
- Jose Luis Campo Rivas
- Rubén Castillo García

## Introducción

Nuestra aplicación,GameGuru, es un sistema de recomendación basado en contenido y en sentimientos que recomienda diferentes juegos en base a descripciones , siendo estas influenciadas por la valoración de los juegos. Nuestro sistema tiene 4 formas de funcionar.
- 1ª Le puedes pedir que te recomiende juegos parecidos dado uno como base
- 2ª Le puedes dar una descripción para que te recomiende juegos basados en esa descripción
- 3ª Te permite seleccionar un juego y realizar una valoración entre 1 y 3 ⭐
- 4ª Le puedes dar una reseña y a partir de esta el sistema te creará una puntuación acorde a la reseña dada, siendo esta entre 1 y 3 ⭐


## Funcionamiento
Para poder ejecutar el menú y funcione todo correctamente, es necesario ejecutar el código de Imports y cada pieza de código con las funciones.

## Imports

A continuación se realizarán la importación de los diferentes elementos necesarios. Primero se importa panda para el tratamiento de los datasets, a continuación el dataset de los juegos y el dataset de los comentarios. A partir de ahí se importa todo lo necesario para el sistema basado en contenidos y sentimientos.

In [ ]:


import pandas as pd

originalData = pd.read_csv('https://raw.githubusercontent.com/alcalde95/abp/main/dataset.csv',delimiter=";")
coments = pd.read_csv('https://raw.githubusercontent.com/alcalde95/abp/main/comentarios.csv',delimiter=";")



from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

import nltk
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Generación previa de valoraciones
A continuación insertamos una nueva columna en el dataset el cual almacenará la puntuación de cada videojuego. Esta puntuación se obtiene como resultado de la división del número de valoraciones de cada videojuego con el resultado de sumar el valor de todas las valoraciones. Estos dos datos se obtienen de un pequeño dataset que hemos generado.

In [ ]:
#Creamos el dataset con algunos juegos y el número de valoraciones junto con la suma total de las valoraciones

df = pd.DataFrame([['Hades', 2, 3,], ['Dead Cells', 5,10], ['Risk of Rain 2', 8,24]],
     index=[0, 1, 2],
     columns=['juego', 'numeroValoraciones','sumaTotalValoraciones'])

#Insertamos una nueva columna en el dataset. Al principio no hay videojuegos con valoraciones

originalData['valorValoracion'] = 0

#Insertamos el valor de las puntuaciones a los siguientes juegos

originalData.loc[originalData['nombre']=='Hades', 'valorValoracion'] = df.loc[df['juego']=='Hades','sumaTotalValoraciones'].values[0] / df.loc[df['juego']=='Hades','numeroValoraciones'].values[0]
originalData.loc[originalData['nombre']=='Dead Cells', 'valorValoracion'] = df.loc[df['juego']=='Dead Cells','sumaTotalValoraciones'].values[0] / df.loc[df['juego']=='Dead Cells','numeroValoraciones'].values[0]
originalData.loc[originalData['nombre']=='Risk of Rain 2', 'valorValoracion'] = df.loc[df['juego'] =='Risk of Rain 2','sumaTotalValoraciones'].values[0] / df.loc[df['juego']=='Risk of Rain 2','numeroValoraciones'].values[0]
originalData.loc[originalData['nombre']=='Zelda: A Link Between Worlds', 'valorValoracion'] = 2



## Sistema de recomendación basado en juegos del dataset

A continuación podemos ver la función encargada de realizar la recomendación de juegos basados en la descripción de un juego previamente almacenado en el dataset.
En esta función, primero se realiza una **tokenization**, que consiste en la división de las descripciones en palabras. A continuación se eliminan aquellas palabras que se consideran sin valor, denominadas **stopwords** y tras esto se extrae la raíz de las palabras, llamado **stemmization**.

Una vez completado estos tres procesos se crea el **bag-of-words** utilizando TF-IDF y así obtener un array con vectores de frecuencias. Utilizamos la distancia coseno para obtener la distancia con el juego seleccionado.

Tras obtener los resultados anteriores, ordenamos basándonos en aquellos que tienen un mayor acierto, le pedimos al usuario que nos indique cuántos juegos quiere mostrar y se los mostramos, mostrando juego y la distancia del juego mostrado al pedido.

In [ ]:
def recomendadorContenido():
  ps = PorterStemmer()

  preprocessedText = []

  for row in originalData.itertuples():

      text = word_tokenize(row[9]) ## indice de la columna que contiene el texto
      ## Se remueven las stopwords

      stops = set(stopwords.words("spanish"))

      text = [ps.stem(w) for w in text if not w in stops and w.isalnum()]
      text = " ".join(text)

      preprocessedText.append(text)

  ## Se añade una nueva columna al dataset con todos los textos procesados
  preprocessedData = originalData
  preprocessedData['processed_text'] = preprocessedText

  ## Se crea la bolsa de palabras
  from sklearn.feature_extraction.text import TfidfVectorizer

  bagOfWordsModel = TfidfVectorizer()
  bagOfWordsModel.fit(preprocessedData['processed_text'])
  textsBoW= bagOfWordsModel.transform(preprocessedData['processed_text'])

  ## Se crea la matriz de distancias
  from sklearn.metrics import pairwise_distances

  distance_matrix= pairwise_distances(textsBoW,textsBoW ,metric='cosine')

  searchTitle = "Hades" #Juego base para las recomendaciones
  indexOfTitle = preprocessedData[preprocessedData['nombre']==searchTitle].index.values[0]

  distance_scores = list(enumerate(distance_matrix[indexOfTitle]))

  ordered_scores = sorted(distance_scores, key=lambda x: x[1])
  ## Se piden cuántos juegos quiere mostrar
  range=int(input("Introduzca cuantos juegos desea ver: "))
  top_scores = ordered_scores[1:range+1]

  top_indexes = [i[0] for i in top_scores]

  puntuaciones = [valor for _, valor in top_scores]

   # Obtener las filas de preprocessedData correspondientes a top_indexes
  filas_seleccionadas = originalData.loc[top_indexes, ['nombre', 'valorValoracion']]

  #Una vez tenemos las filas seleccionadas, las ordenamos por en función de su valoración

  filas_seleccionadas = filas_seleccionadas.sort_values('valorValoracion',ascending =False)

  #Teniendo las filas ordenadas, tenemos que ordenar las puntuaciones para que así queden ordenados siguiendo la nueva organización de filas_seleccionadas

  puntuaciones = [puntuaciones[top_indexes.index(idx)] for idx in filas_seleccionadas.index]


  # Mostrar cada fila junto con el valor correspondiente de puntuaciones
  for i, (index, (nombre, valor_valoracion)) in enumerate(filas_seleccionadas.iterrows()):
    print(f"{nombre} - Puntuación: {puntuaciones[i]} - Valor Valoración: {valor_valoracion}")

## Sistema de recomendación basada en la descripción dada por el usuario

Esta función es prácticamente igual a la anterior, lo que las diferencian es que para poder realizar una recomendación basada en una descripción dada por el usuario, se crea una nueva entrada en el dataset con la descripción, se realizan todos los procesos y a la hora de mostrar los juegos se hacen en relación con la última entrada del dataset que es la introducida la final.

Una vez calculado todo, para que quede el dataset limpio se elimina la última fila del dataset.

In [ ]:
def recomendadorContenidoDescripcion():
  ps = PorterStemmer()
  global originalData
  preprocessedText = []
  ## Se crea e inserta la nueva fila
  nueva_fila = {
    'id': len(originalData),
    'nombre': '',
    'desarrollador': '',
    'jugadores': '',
    'genero': '',
    'PEGI': '',
    'plataforma': '',
    'ano_salida': '',
    'descripcion': str(input("Introduzca la descripcion del juego: ")),

  }


  indice_nueva_fila = len(originalData)


  originalData.loc[indice_nueva_fila] = nueva_fila

  originalData.loc[indice_nueva_fila]

  for row in originalData.itertuples():

      text = word_tokenize(row[9]) ## indice de la columna que contiene el texto
      ## Remove stop words

      stops = set(stopwords.words("spanish"))

      text = [ps.stem(w) for w in text if not w in stops and w.isalnum()]
      text = " ".join(text)

      preprocessedText.append(text)



  preprocessedData = originalData
  preprocessedData['processed_text'] = preprocessedText
  from sklearn.feature_extraction.text import TfidfVectorizer

  bagOfWordsModel = TfidfVectorizer()
  bagOfWordsModel.fit(preprocessedData['processed_text'])
  textsBoW= bagOfWordsModel.transform(preprocessedData['processed_text'])

  from sklearn.metrics import pairwise_distances

  distance_matrix= pairwise_distances(textsBoW,textsBoW ,metric='cosine')


  indexOfTitle = indice_nueva_fila

  distance_scores = list(enumerate(distance_matrix[indexOfTitle]))

  ordered_scores = sorted(distance_scores, key=lambda x: x[1])

  range=int(input("Introduzca cuantos juegos desea ver: "))
  top_scores = ordered_scores[1:range+1]

  top_indexes = [i[0] for i in top_scores]

  ## Una vez calculado todo, se elimina la última fila
  originalData = originalData.drop(1001, axis=0).reset_index(drop=True)


  puntuaciones = [valor for _, valor in top_scores]



  # Obtener las filas de preprocessedData correspondientes a top_indexes
  filas_seleccionadas = originalData.loc[top_indexes, ['nombre', 'valorValoracion']]

  #Una vez tenemos las filas seleccionadas, las ordenamos por en función de su valoración

  filas_seleccionadas = filas_seleccionadas.sort_values('valorValoracion',ascending =False)

  puntuaciones = [puntuaciones[top_indexes.index(idx)] for idx in filas_seleccionadas.index]

  # Mostrar cada fila junto con el valor correspondiente de puntuaciones
  for i, (index, (nombre, valor_valoracion)) in enumerate(filas_seleccionadas.iterrows()):
    print(f"{nombre} - Puntuación: {puntuaciones[i]} - Valor Valoración: {valor_valoracion}")

# Sistema de valoración basado en sentimientos

Este cuenta con 2 modos, una valoración manual, donde tu puedes poner valoración sólo o puedes indicarle un comentario y que él te genere una valoración en específico


Generar la bolsa de palabras del dataset de comentarios

In [ ]:
ps = PorterStemmer();
preprocessedComents = []

for row in coments.itertuples():
  coment = word_tokenize(row[1]) ##Indice de la columna que contiene el comentario
  ## Eliminar palabras vacias (stopwords)
  stops = set(stopwords.words("spanish"))
  coment = [ps.stem(w) for w in coment if not w in stops and w.isalnum()]
  coment = " ".join(coment)

  preprocessedComents.append(coment)

preprocessedData = coments
preprocessedData['processed_coment'] = preprocessedComents

from sklearn.feature_extraction.text import TfidfVectorizer

bagOfWordsModel = TfidfVectorizer()
bagOfWordsModel.fit(preprocessedData['processed_coment'])
textsBoW = bagOfWordsModel.transform(preprocessedData['processed_coment'])

Entrenamiento del algoritmo de clasificación

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import svm

X_train, X_temp, y_train, y_temp = train_test_split(textsBoW, preprocessedData['puntuation'], test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
#Modelo de Clasificacion
svc = svm.SVC(kernel='linear')
#Entrenamiento
svc.fit(X_train, y_train)
#Evaluacion del modelo en conj. de validacion
accuracy_val = svc.score(X_val,y_val)
print(f'Accuracy en el conjunto de validacion: {accuracy_val}')
#Evaluacion final en el conjunto de prueba
accuracy_test = svc.score(X_test, y_test)
print(f'Accuracy en el conjunto de prueba: {accuracy_test}')

Accuracy en el conjunto de validacion: 1.0
Accuracy en el conjunto de prueba: 1.0


## Funciones creadas a partir del sistema algoritmo de clasificación

In [ ]:
def sistemaValoracion():
  print("1:Valoracion Manual")
  print("2:Valoracion Automatica")
  opcion = int(input())

  match opcion:
    case 1:valoracionManual()
    case 2:valoracionAutomatica()

In [ ]:
def valoracionAutomatica():

  print("Introduzca el número del juego que quiere valorar")
  print(originalData[["id","nombre"]])
  input()

  print("* = Malo")
  print("** = Neutro")
  print("*** = Muy bueno")

  user_comment = input("Introduce tu comentario: ")
  #Preprocesamiento del comentario de entrada
  preprocessed_user_comment = word_tokenize(user_comment)
  preprocessed_user_comment = [ps.stem(w) for w in preprocessed_user_comment if not w in stops and w.isalnum()]
  preprocessed_user_comment = " ".join(preprocessed_user_comment)
  #Transformacion del comentario usando modelo TF-IDF
  user_comment_vector = bagOfWordsModel.transform([preprocessed_user_comment])
  #Predicción de puntuacion usando modelo SVM
  predicted_punctuation = svc.predict(user_comment_vector)
  match predicted_punctuation:
    case 1:stars = "*"
    case 2:stars = "**"
    case 3:stars = "***"
  print("La valoracion predicha para el comentario es ",stars)

In [ ]:
def valoracionManual():

  print("Introduzca el número del juego a  valorar")
  print(originalData[["id","nombre"]])
  input()

  print("* = Malo")
  print("** = Neutro")
  print("*** = Muy bueno")

  input()

## Menu:

Este es un menú para poder acceder a todas las funcionalidades mostradas.

Introduzca 0,1,2 o 3 en función de la funcionalidad a la que quiere acceder.

In [ ]:
opcion = 1

while opcion != 0:
  print("0:Exit")
  print("1:Recomendador de Contenido")
  print("2:Recomendador de Contenido según una descripción")
  print("3:Sistema de Valoracion")
  opcion = int(input())

  match opcion:
    case 1:recomendadorContenido()
    case 2:recomendadorContenidoDescripcion()
    case 3:sistemaValoracion()
## añadir que se muestre el % de acie22rto en la recomendación de contenido


0:Exit
1:Recomendador de Contenido
2:Recomendador de Contenido según una descripción
3:Sistema de Valoracion
1
Introduzca cuantos juegos desea ver: 5
Risk of Rain 2 - Puntuación: 0.8137709815142152 - Valor Valoración: 3.0
Dead Cells - Puntuación: 0.8076785529305041 - Valor Valoración: 2.0
Hades - Puntuación: 0.694234112605255 - Valor Valoración: 1.5
Hades - Puntuación: 0.694234112605255 - Valor Valoración: 1.5
Sparklite - Puntuación: 0.8261883600724826 - Valor Valoración: 0.0
0:Exit
1:Recomendador de Contenido
2:Recomendador de Contenido según una descripción
3:Sistema de Valoracion
2
Introduzca la descripcion del juego: zelda
Introduzca cuantos juegos desea ver: 5
Zelda: A Link Between Worlds - Puntuación: 0.6772971985261917 - Valor Valoración: 2.0
Zelda: Twilight Princess - Puntuación: 0.43575362240240134 - Valor Valoración: 0.0
The Legend of Zelda: Breath of the Wild - Puntuación: 1.0 - Valor Valoración: 0.0
Red Dead Redemption 2 - Puntuación: 1.0 - Valor Valoración: 0.0
The Witcher

# Transformación a PDF

In [ ]:
!pip install pdfkit
!apt-get update


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [634 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,599 kB]
Get:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.

In [ ]:
from google.colab import drive
import shutil

# Monta Google Drive (puedes omitir esto si no necesitas guardar el PDF en Google Drive)
drive.mount('/content/drive')

# Especifica el nombre del notebook y el nombre del archivo PDF de salida
notebook_name = '/content/drive/My Drive/Colab Notebooks/GameGuru.ipynb'
output_pdf = '/content/drive/My Drive/Colab Notebooks/GameGuru.ipynb.pdf'  # Cambia la ruta según sea necesario

!apt-get install texlive-xetex

# Exporta el notebook a PDF utilizando nbconvert
!jupyter nbconvert --to pdf "$notebook_name"

# Copia el PDF a la ubicación deseada en Google Drive
shutil.copyfile(notebook_name.replace('.ipynb', '.pdf'), output_pdf)




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
texlive-xetex is already the newest version (2021.20220204-1).
0 upgraded, 0 newly installed, 0 to remove and 28 not upgraded.
[NbConvertApp] Converting notebook /content/drive/My Drive/Colab Notebooks/GameGuru.ipynb to pdf
[NbConvertApp] Writing 53639 bytes to notebook.tex
[NbConvertApp] Building PDF
[NbConvertApp] Running xelatex 3 times: ['xelatex', 'notebook.tex', '-quiet']
[NbConvertApp] Running bibtex 1 time: ['bibtex', 'notebook']
[NbConvertApp] WARNING | bibtex had problems, most likely because there were no citations
[NbConvertApp] PDF successfully created
[NbConvertApp] Writing 62996 bytes to /content/drive/My Drive/Colab Notebooks/GameGuru.pdf


'/content/drive/My Drive/Colab Notebooks/GameGuru.ipynb.pdf'

## Bibliografía:
[https://github.com/adrseara/abp_notebooks/blob/master/Tutorial_Recomendador_basado_en_contenido.ipynb](https://github.com/adrseara/abp_notebooks/blob/master/Tutorial_Recomendador_basado_en_contenido.ipynb)
[https://github.com/adrseara/abp_notebooks/blob/master/Tutorial_análisis_de_sentimientos.ipynb](https://github.com/adrseara/abp_notebooks/blob/master/Tutorial_análisis_de_sentimientos.ipynb)
